In [24]:
#get scores 

import os
import subprocess
import json

# Define paths
translations_dir = "translations_gpt35_xx_eng2"
reference_file = "reference_xx_eng/eng_Latn.txt"

# Dictionary to store scores
scores = {}

# Loop over all .txt files in the translations directory
for file in os.listdir(translations_dir):
    if file.endswith(".txt"):  # Process only .txt files
        language_name = file.replace(".txt", "")  # Extract language name
        translation_file = os.path.join(translations_dir, file)  # Full path

        # Run sacrebleu command
        command = f"sacrebleu -m chrf --chrf-word-order 2 {translation_file} < {reference_file}"
        result = subprocess.run(command, shell=True, capture_output=True, text=True)

        # Extract the score from the output
        try:
            output_json = json.loads(result.stdout)  # Parse JSON output
            scores[language_name] = output_json["score"]  # Store score
        except json.JSONDecodeError:
            print(f"Error processing {language_name}: Invalid JSON output")
        except KeyError:
            print(f"Error: 'score' field not found in {language_name} output")

# Print results
print(scores)


{'bjn_Latn': 47.1, 'lus_Latn': 31.1, 'pbt_Arab': 33.2, 'spa_Latn': 55.5, 'som_Latn': 37.4, 'pes_Arab': 56.7, 'gle_Latn': 55.4, 'szl_Latn': 58.4, 'zho_Hans': 53.5, 'kam_Latn': 26.8, 'dyu_Latn': 22.1, 'zul_Latn': 40.7, 'ron_Latn': 64.7, 'nno_Latn': 66.8, 'san_Deva': 37.3, 'xho_Latn': 40.0, 'hye_Armn': 40.5, 'hun_Latn': 59.3, 'lao_Laoo': 25.8, 'urd_Arab': 51.4, 'slv_Latn': 59.4, 'pag_Latn': 42.8, 'zho_Hant': 52.5, 'tzm_Tfng': 17.8, 'nob_Latn': 66.0, 'gaz_Latn': 22.0, 'isl_Latn': 55.6, 'arb_Arab': 60.5, 'mri_Latn': 41.7, 'sun_Latn': 49.7, 'asm_Beng': 38.8, 'yue_Hant': 54.3, 'hrv_Latn': 61.5, 'lug_Latn': 29.6, 'bem_Latn': 28.8, 'grn_Latn': 30.6, 'khm_Khmr': 30.6, 'taq_Latn': 24.6, 'ckb_Arab': 30.7, 'afr_Latn': 74.7, 'mos_Latn': 22.9, 'min_Arab': 19.9, 'lin_Latn': 30.7, 'sna_Latn': 33.4, 'cym_Latn': 68.7, 'ory_Orya': 42.1, 'ita_Latn': 58.0, 'ban_Latn': 45.5, 'zsm_Latn': 64.5, 'bod_Tibt': 16.9, 'ibo_Latn': 27.8, 'kaz_Cyrl': 42.6, 'jpn_Jpan': 52.5, 'smo_Latn': 36.1, 'npi_Deva': 51.4, 'snd_Arab

In [21]:
df = pd.read_csv('baseline_result - mt_all.csv')

In [22]:
scores2=df.set_index('language_code')['gpt_35_chrf'].to_dict()

In [31]:
import pandas as pd
language_mapping = pd.read_csv('language_mapping.csv')
language_mapping_index = language_mapping.set_index('FLORES-200 code')['Language'].to_dict()
# Convert to DataFrame
all_languages = list(language_mapping_index.keys())
all_languages.remove('eng_Latn')

all_languages2 = list(language_mapping_index.values())
all_languages2.remove('English')

df = pd.DataFrame({
    "Language": all_languages2,
    "FLORES-200 code": all_languages,
    "xx - English": [scores[lang] for lang in all_languages],
    "English - xx": [scores2[lang] for lang in all_languages]
})

# Save to CSV
df.to_csv("flores_gpt35_results.csv", index=False)






In [32]:
import pandas as pd
language_mapping = pd.read_csv('language_mapping.csv')
language_mapping_index = language_mapping.set_index('FLORES-200 code')['Language'].to_dict()
flores_languages = list(language_mapping_index.keys())

# Convert to DataFrame
all_languages = list(language_mapping_index.keys()) #code

all_languages2 = list(language_mapping_index.values()) #name

import bisect
# New elements to insert
new_key = "nqo_Nkoo"
new_value = "N'ko"

# Find the correct index to insert while maintaining sorted order
index = bisect.bisect(all_languages, new_key)

# Insert in both lists at the correct position
all_languages.insert(index, new_key)
all_languages2.insert(index, new_value)

In [33]:
flores_df = pd.read_csv('flores_gpt35_results.csv')
old_results_df = pd.read_csv('old_results.csv')
sib_df = pd.read_csv('sib_gpt35_results_april2025_prompt2.csv')

lang_family = old_results_df.set_index('language_code')['language_family'].to_dict()
lang_family_array = []
for lang in all_languages:
    if lang in lang_family.keys():
        lang_family_array.append(lang_family[lang])
    else:
        lang_family_array.append('')

sib_scores = sib_df.set_index('Language code')['Accuracy'].to_dict()
sib_array = []
for lang in all_languages:
    if lang in sib_scores.keys():
        sib_array.append(sib_scores[lang])
    else:
        sib_array.append('')

flores_xx_eng_scores = flores_df.set_index('FLORES-200 code')['xx - English'].to_dict()
flores_xx_eng_array = []
for lang in all_languages:
    if lang in flores_xx_eng_scores.keys():
        flores_xx_eng_array.append(flores_xx_eng_scores[lang])
    else:
        flores_xx_eng_array.append('')

flores_eng_xx_scores = flores_df.set_index('FLORES-200 code')['English - xx'].to_dict()
flores_eng_xx_array = []
for lang in all_languages:
    if lang in flores_eng_xx_scores.keys():
        flores_eng_xx_array.append(flores_eng_xx_scores[lang])
    else:
        flores_eng_xx_array.append('')
        
result_df = pd.DataFrame({
    "language_name": all_languages2,
    "language_code": all_languages,
    "language_family": lang_family_array,
    "SIB GPT-3.5-Turbo": sib_array,
    "Flores xx-eng GPT-3.5-Turbo": flores_xx_eng_array,
    "Flores eng-xx GPT-3.5-Turbo": flores_eng_xx_array

})

result_df.to_csv('overall_results_gpt35.csv',index=False)


In [34]:
df2 = pd.read_csv('overall_results_gpt35.csv')
z=df2.groupby('language_family') \
       .agg({'language_name':'size','SIB GPT-3.5-Turbo':'mean','Flores xx-eng GPT-3.5-Turbo':'mean','Flores eng-xx GPT-3.5-Turbo':'mean'} ) \
       .rename(columns={'language_name':'count'}) \
       .sort_values(by=['count'], ascending=False) \
       .reset_index().round(1)
z.to_csv('overall_by_family_gpt35.csv',index=False)


In [5]:
df1 = pd.read_csv('flores_gpt4o_results_april_2025_prompt2.csv')

In [8]:
flores_languages= list(df1['FLORES-200 code'])

In [9]:
len(flores_languages)

203

In [10]:
df2 = pd.read_csv('sib_gpt4o_results_april2025_prompt2.csv')
sib_languages = list(df2['Language code'])
len(sib_languages)

205

In [14]:
df3 = pd.read_csv('old_results.csv')
sib_languages_old = list(df3['language_code'])
sib_languages_old.remove('ave')
sib_languages_old.remove('std')
len(sib_languages_old)

205

In [15]:
sib_languages_old == sib_languages

False

In [18]:
set(sib_languages) - set(flores_languages)

{'eng_Latn', 'nqo_Nkoo'}

In [19]:
set(flores_languages) - set(sib_languages)

set()